<a href="https://colab.research.google.com/github/just99kim/CS598/blob/main/DL4H_Team_121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video Link

[Video Link](https://drive.google.com/file/d/1cupg5eDzopz26ZIspRYMFvHcQ1ZHorpC/view?usp=sharing)

# Introduction

*   Background
  * Alzheimer’s Disease (AD) is the most common neurodegenerative disorder, characterized by complex pathogenesis that complicates early diagnosis. Accurate early diagnosis, particularly distinguishing MCI from AD, is crucial for delaying disease progression through timely intervention.
*   Paper explanation
  * This paper seeks to build upon the groundbreaking work presented in the development of the Multimodal Alzheimer’s Disease Diagnosis framework (MADDi), an attention-based deep learning model designed for the accurate diagnosis of Alzheimer's Disease (AD) through the integration of imaging, genetic, and clinical data [1]. The original study demonstrates the effectiveness of this model, achieving state-of-the-art performance with an accuracy of 96.88% in distinguishing between control, Mild Cognitive Impairment (MCI), and Alzheimer’s Disease.

# Scope of Reproducibility:

I will test the following hypothesis from the paper.

1. MADDi classifies MCI, AD, and controls with significant accuracy on a held-out test set.
2. MADDi classifies MCI, AD, and controls with higher accuracy when compared to unimodal models.

# Methodology



  * Source of the data: The data is collected from the Alzheimer's Disease Neuroimaging Initiative (ADNI) dataset. The data was not provided in this repository and needs to be requested directly from the ADNI [here](https://adni.loni.usc.edu/data-samples/access-data/).
  * Data Preprocessing: A list of data patient IDs with their diagnoses were created. Clinical, imaging and genetic data were preprocessed accordingly.
  * Training & Valuation: A uni-modal model baseline (i.e. clinical, genetic and images) and multimodal architecture were trained and evaluated separately.

##  Environment

Python 3.7.4 (and above)
Tensorflow 2.6.0 are required to run this notebook.

Further details on all package requirements used in this repository can be found below



In [ ]:
absl-py==0.14.1
aiohttp==3.8.1
aiosignal==1.2.0
astunparse==1.6.3
async-timeout==4.0.1
asynctest==0.13.0
attrs==21.2.0
cached-property==1.5.2
cachetools==4.2.4
certifi==2021.10.8
charset-normalizer==2.0.6
clang==5.0
click==8.0.3
cycler==0.10.0
datasets==1.15.1
dill==0.3.4
filelock==3.4.0
flatbuffers==1.12
frozenlist==1.2.0
fsspec==2021.11.0
gast==0.4.0
google-auth==1.35.0
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
grpcio==1.41.0
h5py==3.1.0
idna==3.2
imageio==2.9.0
importlib-metadata==4.8.1
joblib==1.1.0
keras==2.6.0
Keras-Preprocessing==1.1.2
kiwisolver==1.3.2
Markdown==3.3.4
matplotlib==3.4.3
multidict==5.2.0
multiprocess==0.70.12.2
networkx==2.6.3
nibabel==3.2.1
numpy==1.19.5
oauthlib==3.1.1
opt-einsum==3.3.0
packaging==21.0
pandas==1.3.3
pickle5==0.0.12
Pillow==8.3.2
protobuf==3.18.1
pyarrow==6.0.1
pyasn1==0.4.8
pyasn1-modules==0.2.8
pyparsing==2.4.7
python-dateutil==2.8.2
pytz==2021.3
PyWavelets==1.1.1
PyYAML==6.0
regex==2021.11.10
requests==2.26.0
requests-oauthlib==1.3.0
rsa==4.7.2
sacremoses==0.0.46
scikit-image==0.18.3
scikit-learn==1.0
scipy==1.7.1
six==1.15.0
tensorboard==2.6.0
tensorboard-data-server==0.6.1
tensorboard-plugin-wit==1.8.0
tensorflow==2.6.0
tensorflow-addons==0.14.0
tensorflow-estimator==2.6.0
termcolor==1.1.0
threadpoolctl==3.0.0
tifffile==2021.8.30
tokenizers==0.10.3
tqdm==4.62.3
typeguard==2.12.1
typing-extensions==3.7.4.3
urllib3==1.26.7
Werkzeug==2.0.2
wrapt==1.12.1
xxhash==2.0.2
yarl==1.7.2
zipp==3.6.0

##  Data
Data includes raw data (clinical, genetic, images) and preprocessing.



# Downloading Data

Data was downlaoded from the Alzheimer's Disease Neuroimaging Initiative (ADNI) dataset. The data was not provided in this repository and needs to be requested directly from the ADNI here.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Create a list of patient IDs with their diagnosis,**

This method take diagnosis from images, clinical, and diagnosis sheet, and creates one ground truth (where all three agree) and one majority vote (where two agree) diagnosis files.

In [ ]:
import pandas as pd
import math
clinical = pd.read_csv("/content/drive/My Drive/ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})
#this file is the metadata file that one can get from downloading MRI images from ADNI
img = pd.read_csv("/content/drive/My Drive/MRILIST_07May2024.csv")
comb = pd.read_csv("/content/drive/My Drive/DXSUM_PDXCONV.csv")[["RID", "PTID" , "PHASE"]]

In [ ]:
def read_diagnose(file_path: str = '/content/drive/My Drive/DXSUM_PDXCONV.csv', verbose=False):
    # Read diagnostic summary
    diagnostic_summary = pd.read_csv(file_path, index_col='PTID')
    diagnostic_summary = diagnostic_summary.sort_values(by=["update_stamp"], ascending=True)
    # Create dictionary
    diagnostic_dict: dict = {}
    for key, data in diagnostic_summary.iterrows():
        # Iterate for each row of the document
        phase: str = data['PHASE']
        diagnosis: float = -1.
        if phase == "ADNI1":
            diagnosis = data['DIAGNOSIS']
        elif phase == "ADNI2" or phase == "ADNIGO":
            dxchange = data['DIAGNOSIS']
            if dxchange == 1 or dxchange == 7 or dxchange == 9:
                diagnosis = 1.
            if dxchange == 2 or dxchange == 4 or dxchange == 8:
                diagnosis = 2.
            if dxchange == 3 or dxchange == 5 or dxchange == 6:
                diagnosis = 3.
        elif phase == "ADNI3":
            diagnosis = data['DIAGNOSIS']
        else:
            print(f"ERROR: Not recognized study phase {phase}")
            exit(1)
        # Update dictionary
        if not math.isnan(diagnosis):
            diagnostic_dict[key] = diagnosis
    if verbose:
        print_diagnostic_dict_summary(diagnostic_dict)
    return diagnostic_dict


def print_diagnostic_dict_summary(diagnostic_dict: dict):
    print(f"Number of diagnosed patients: {len(diagnostic_dict.items())}\n")
    n_NL = 0
    n_MCI = 0
    n_AD = 0
    for (key, data) in diagnostic_dict.items():
        if data == 1:
            n_NL += 1
        if data == 2:
            n_MCI += 1
        if data == 3:
            n_AD += 1
    print(f"Number of NL patients: {n_NL}\n"
          f"Number of MCI patients: {n_MCI}\n"
          f"Number of AD patients: {n_AD}\n")

In [ ]:
d = read_diagnose()
print_diagnostic_dict_summary(d)

In [ ]:
new = pd.DataFrame.from_dict(d, orient='index').reset_index()

In [ ]:
clinical.head()

In [ ]:
clinical["year"] = clinical["EXAMDATE"].str[:4]

In [ ]:
clinical["Subject"] = clinical["SUBJECT_KEY"].str.replace("ADNI_", "").str.replace("s", "S")

In [ ]:
clinical.rename(columns={'Phase': 'PHASE'}, inplace=True)
comb.rename(columns={'phase': 'PHASE'}, inplace=True)

c = comb.merge(clinical, on = ["RID", "PHASE"])



In [ ]:
c = c.drop("Subject", axis =1)

In [ ]:
c = c.rename(columns = {"PTID":"Subject"})

In [ ]:
img["year"] = img["SCANDATE"].str[5:].str.replace("/", "")

In [ ]:
img = img.replace(["CN", "MCI", "AD"], [ 0, 1, 2])

In [ ]:
c["DX"] = c["DX"] -1

In [ ]:
new[0] = new[0].astype(int) -1

In [ ]:
new = new.rename(columns = {"index":"Subject", 0:"GroupN"})

In [ ]:
new.rename(columns={'Subject': 'SUBJECT'}, inplace=True)
img.rename(columns={'SUBJECT': 'SUBJECT'}, inplace=True)
c.rename(columns={'Subject': 'SUBJECT'}, inplace=True)


m = new.merge(c, on = 'SUBJECT', how = "outer").merge(img, on = 'SUBJECT', how = "outer")

In [ ]:
print(m.columns)

m[["GroupN", "DX", "RID"]]

In [ ]:
m = m[["SUBJECT", "GroupN", "RID", "DX", "PHASE"]].drop_duplicates()

In [ ]:
m = m.dropna(subset = ["GroupN", "RID", "DX"], how="all").drop_duplicates()
m

In [ ]:
m.loc[m["DX"].isna() & m["RID"].isna(), "RID"] = m.loc[m["DX"].isna() & m["RID"].isna(), "GroupN"]
m.loc[m["DX"].isna() & m["RID"].isna(), "DX"] = m.loc[m["DX"].isna() & m["RID"].isna(), "GroupN"]

In [ ]:
m1 = m[m["GroupN"] == m["RID"]]
m3 = m[m["GroupN"] == m["DX"]]
m4 = m[m["RID"] == m["DX"]]
m2 = m1[m1["RID"] == m1["DX"]]

In [ ]:
m1 = m1[["SUBJECT", "GroupN", "RID", "DX", "PHASE"]]
m1

In [ ]:
m1.loc[m1["DX"].isna(), "DX"] = m1.loc[m1["DX"].isna(), "RID"]

In [ ]:
m3 = m3[["SUBJECT", "GroupN", "RID", "DX", "PHASE"]]
m3

In [ ]:
m3.loc[m3["RID"].isna(), "RID"] = m3.loc[m3["RID"].isna(), "GroupN"]

In [ ]:
m4 = m4[["SUBJECT", "GroupN", "RID", "DX", "PHASE"]]
m4

In [ ]:
m4[m4["GroupN"] != m4["DX"]]

In [ ]:
m2[["SUBJECT", "GroupN", "RID", "DX", "PHASE"]]

In [ ]:
m5 = pd.concat([m1,m3,m4])
i = m5[m5["RID"] == m5["GroupN"]]
i = i[i["RID"] == i["DX"]]

In [ ]:
i = i.drop_duplicates()

In [ ]:
i

In [ ]:
i[["SUBJECT", "RID", "PHASE"]].to_csv("ground_truth.csv")

In [ ]:
m.update(m5[~m5.index.duplicated(keep='first')])

In [ ]:
indexes = m.index

In [ ]:
#if none of the three diagnosis agree, then we set the value to -1
m["GROUP"] = -1

In [ ]:
for i in indexes:
    row = m.loc[i]
    if (row["GroupN"] == row["RID"]):
        val = row["GroupN"]

        m.loc[i, "GROUP"] = val
    elif (row["GroupN"] == row["DX"]):
        val = row["GroupN"]
        m.loc[i, "GROUP"] = val

    elif (row["RID"] == row["DX"]):
        val = row["Group"]
        m.loc[i, "GROUP"] = val

In [ ]:
m5 = m5[~m5.index.duplicated(keep='first')]
m5

In [ ]:
m[m["GROUP"] != -1]

In [ ]:
m[["SUBJECT", "GroupN", "RID", "DX", "GROUP", "PHASE"]].to_csv("diagnosis_full.csv")

**Preprocessing Clinical Data**

This series of methods will preprocess the clinical data and create a CSV file with the necessary data.

In [ ]:
#Import necesary packages

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout,MaxPooling1D, Flatten,BatchNormalization, GaussianNoise,Conv1D
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, save_model, load_model


In [ ]:
#this was created in the first step
diag = pd.read_csv("ground_truth.csv").drop("Unnamed: 0", axis=1)

Below we are combining several clinical datasets.



In [ ]:
demo = pd.read_csv("PTDEMOG.csv")

In [ ]:
neuro = pd.read_csv("NEUROEXM.csv")
neuro.columns

In [ ]:
clinical = pd.read_csv("ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})

In [ ]:
clinical.head()

In [ ]:
diag["Subject"].value_counts()


In [ ]:
comb = pd.read_csv("DXSUM_PDXCONV_ADNIALL.csv")[["RID", "PTID" , "Phase"]]


In [ ]:
m = comb.merge(demo, on = ["RID", "Phase"]).merge(neuro,on = ["RID", "Phase"]).merge(clinical,on = ["RID", "Phase"]).drop_duplicates()

In [ ]:
m.columns = [c[:-2] if str(c).endswith(('_x','_y')) else c for c in m.columns]
m = m.loc[:,~m.columns.duplicated()]


In [ ]:
diag = diag.rename(columns = {"Subject": "PTID"})


In [ ]:
m = m.merge(diag, on = ["PTID", "Phase"])

In [ ]:
m["PTID"].value_counts()


In [ ]:
t = m


In [ ]:
t = t.drop(["ID",  "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2",
            "update_stamp",  "PTSOURCE", "PTDOBMM","DX"], axis=1)

In [ ]:
t.columns

In [ ]:
t = t.fillna(-4)
t = t.replace("-4", -4)
cols_to_delete = t.columns[(t == -4).sum()/len(t) > .70]
t.drop(cols_to_delete, axis = 1, inplace = True)

In [ ]:
len(t.columns)

In [ ]:
t["PTWORK"] = t["PTWORK"].str.lower().str.replace("housewife", "homemaker").str.replace("rn", "nurse").str.replace("bookeeper",


In [ ]:
t["PTWORK"] = t["PTWORK"].fillna("-4").astype(str)


In [ ]:
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*teach.*$)', 'education')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*bookkeep.*$)', 'bookkeeper')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*wife.*$)', 'homemaker')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*educat.*$)', 'education')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*engineer.*$)', 'engineer')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*eingineering.*$)', 'engineer')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*computer programmer.*$)', 'engineer')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*nurs.*$)', 'nurse')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*manage.*$)', 'managment')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*therapist.*$)', 'therapist')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*sales.*$)', 'sales')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*admin.*$)', 'admin')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*account.*$)', 'accounting')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*real.*$)', 'real estate')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*secretary.*$)', 'secretary')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*professor.*$)', 'professor')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*chem.*$)', 'chemist')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*business.*$)', 'business')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*writ.*$)', 'writing')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*psych.*$)', 'psychology')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*analys.*$)', 'analyst')

In [ ]:
cond = t['PTWORK'].value_counts()
threshold = 10
t['PTWORK'] = np.where(t['PTWORK'].isin(cond.index[cond >= threshold ]), t['PTWORK'], 'other')

In [ ]:
categorical = ['PTGENDER', 'PTWORK',
 'PTHOME',
 'PTMARRY',
 'PTEDUCAT',
 'PTPLANG',
 'NXVISUAL',
 'PTNOTRT',
 'NXTREMOR',
 'NXAUDITO',
 'PTHAND']


In [ ]:
quant = ['PTDOBYY',
 'PHC_MEM',
 'PHC_EXF',
 'PTRACCAT',
 'AGE',
 'PTADDX',
 'PTETHCAT',
 'PTCOGBEG',
 'PHC_VSP',
 'PHC_LAN']


In [ ]:
text = ["PTWORK", "CMMED"]


In [ ]:
cols_left = list(set(t.columns) - set(categorical) - set(text)  - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[cols_left]


In [ ]:
for col in cols_left:
    if len(t[col].value_counts()) < 10:
        print(col)
        categorical.append(col)


In [ ]:
to_del = ["PTRTYR", "EXAMDATE", "SUBJECT_KEY", "PTWRECNT"]
t = t.drop(to_del, axis=1)


In [ ]:
quant = list(set(cols_left) - set(categorical) - set(text)  -set(to_del) - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[quant]


In [ ]:
cols_left = list(set(cols_left) - set(categorical) - set(text) - set(quant) - set(to_del))


In [ ]:
#after reviewing the meaning of each column, these are the final ones
l = ['RID', 'PTID', 'Group', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTWORK', 'PTNOTRT', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT',
       'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']


In [ ]:
t[l]

In [ ]:
dfs = []

In [ ]:
for col in categorical:
    dfs.append(pd.get_dummies(t[col], prefix = col))

In [ ]:
cat = pd.concat(dfs, axis=1)

In [ ]:
t[quant]

In [ ]:
cat

In [ ]:
t[["PTID","RID", "Phase", "Group"]]

In [ ]:
c = pd.concat([t[["PTID", "RID", "Phase", "Group"]].reset_index(), cat.reset_index(), t[quant].reset_index()], axis=1).drop("index", axis=1) #tex

In [ ]:
c

In [ ]:
#removing repeating subjects, taking the most recent diagnosis
c = c.groupby('PTID',
                  group_keys=False).apply(lambda x: x.loc[x["Group"].astype(int).idxmax()]).drop("PTID", axis = 1).reset_index(inplace=False)


In [ ]:
c.to_csv("clinical.csv")

In [ ]:
#reading in the overlap test set
ts = pd.read_csv("overlap_test_set.csv").rename(columns={"subject": "PTID"})

#removing ids from the overlap test set
c = c[~c["PTID"].isin(list(ts["PTID"].values))]


In [ ]:
cols = list(set(c.columns) - set(["PTID","RID","subject", "ID","GROUP", "Group", "label", "Phase", "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2", "update_stamp", "DX_x","DX_y", "Unnamed: 0"]))
X = c[cols].values
y = c["Group"].astype(int).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
X_train.to_pickle("X_train_c.pkl")
y_train.to_pickle("y_train_c.pkl")

X_test.to_pickle("X_test_c.pkl")
y_test.to_pickle("y_test_c.pkl")

In [ ]:
# dir and function to load raw data
raw_data_dir = '/content/gdrive/My Drive/Colab Notebooks/<path-to-raw-data>'

def load_raw_data(raw_data_dir):
  # implement this function to load raw data to dataframe/numpy array/tensor
  return None

raw_data = load_raw_data(raw_data_dir)

# calculate statistics
def calculate_stats(raw_data):
  # implement this function to calculate the statistics
  # it is encouraged to print out the results
  return None

# process raw data
def process_data(raw_data):
    # implement this function to process the data as you need
  return None

processed_data = process_data(raw_data)

''' you can load the processed data directly
processed_data_dir = '/content/gdrive/My Drive/Colab Notebooks/<path-to-raw-data>'
def load_processed_data(raw_data_dir):
  pass

'''

**Preprocessing Images Data**

Run the following python script with the directory where images are stored as the argument.

In [ ]:
import numpy as np
import skimage.transform as skTrans
import nibabel as nib
import pandas as pd
import os
import sys
import time


def normalize_img(img_array):
    maxes = np.quantile(img_array,0.995,axis=(0,1,2))
    #print("Max value for each modality", maxes)
    return img_array/maxes


def create_dataset(meta, meta_all,path_to_datadir):
    files = os.listdir(path_to_datadir)
    start = '_'
    end = '.nii'
    for file in files:
        print(file)
        if file != '.DS_Store':
            path = os.path.join(path_to_datadir, file)
            print(path)
            img_id = file.split(start)[-1].split(end)[0]
            idx = meta[meta["Image Data ID"] == img_id].index[0]
            im = nib.load(path).get_fdata()
            n_i, n_j, n_k = im.shape
            center_i = (n_i - 1) // 2
            center_j = (n_j - 1) // 2
            center_k = (n_k - 1) // 2
            im1 = skTrans.resize(im[center_i, :, :], (72, 72), order=1, preserve_range=True)
            im2 = skTrans.resize(im[:, center_j, :], (72, 72), order=1, preserve_range=True)
            im3 = skTrans.resize(im[:, :, center_k], (72, 72), order=1, preserve_range=True)
            im = np.array([im1,im2,im3]).T
            label = meta.at[idx, "Group"]
            subject = meta.at[idx, "Subject"]
            norm_im = normalize_img(im)
            meta_all = meta_all.append({"img_array": im,"label": label,"subject":subject}, ignore_index=True)


    meta_all.to_pickle("mri_meta.pkl")
    meta_all.flush()
    os.fsync(meta_all.fileno())
    time.sleep(0.5)



def main():
    args = sys.argv[1:]
    path_to_meta = args[0]
    path_to_datadir = args[1]
    print(path_to_meta)


    meta = pd.read_csv(path_to_meta)
    print("opened meta")
    print(len(meta))
    #get rid of not needed columns
    meta = meta[["Image Data ID", "Group", "Subject"]] #MCI = 0, CN =1, AD = 2
    meta["Group"] = pd.factorize(meta["Group"])[0]
    #initialize new dataset where arrays will go
    meta_all = pd.DataFrame(columns = ["img_array","label","subject"])
    create_dataset(meta, meta_all, path_to_datadir)

if __name__ == '__main__':
    main()


Use the file created from the script to and use the following notebook to the imaging data into training and testing such that there are no repeating patients in the test set and that the patients in the test set do not appear in training.



In [ ]:
import pandas as pd
import random
#reading in a dataframe that contains image arrays, patient IDs ("subject"), and diagnosis
m2 = pd.read_pickle("mri_meta.pkl")

#cleaning patient IDs
m2["subject"] = m2["subject"].str.replace("s", "S").str.replace("\n", "")

#reading in the overlap test set
ts = pd.read_csv("overlap_test_set.csv")

#removing ids from the overlap test set
m2 = m2[~m2["subject"].isin(list(ts["subject"].values))]

In [ ]:
#there are 551 unique patients
subjects = list(set(m2["subject"].values))
len(subjects)

In [ ]:
0.1*len(m2) #10% for testing

We have 3674 MRI scans from 551 patients (some patients repeated up to 16 times). We selected our testing set such that it has 367 unique MRIs (10% of training) shwon below. We do not allow for any repeating patients in the testing set. We only allowed repetition during training, and no patient was included in both training and testing sets.

In [ ]:
#selecting 367 patient IDs
picked_ids = random.sample(subjects, 367)

In [ ]:
#creating the test set out of the patient IDs
test = pd.DataFrame(columns = ["img_array", "subject", "label"])
for i in range(len(picked_ids)):
    s = m2[m2["subject"] == picked_ids[i]].sample()
    test = test.append(s)

In [ ]:
indexes = list(set(m2.index) - set(test.index))

In [ ]:
#creating the training set using all the other data points
train = m2[m2.index.isin(indexes)]

In [ ]:
train[["img_array"]].to_pickle("img_train.pkl")
test[["img_array"]].to_pickle("img_test.pkl")

In [ ]:
train[["label"]].to_pickle("img_y_train.pkl")
test[["label"]].to_pickle("img_y_test.pkl")

**Preprocessing Genetic Data**

First obtain VCF files from ADNI and then use the vcftools package to filter the files based on your chosen criteria (Hardy-Weinberg equilibrium, genotype quality, minor allele frequency, etc.).

Download Alzheimer's gene list

In [ ]:
filepath = '/content/drive/My Drive/gene_list.csv'


Further filter VCF files according to AD-relrated genges from ALzGene Database (http://www.alzgene.org/)

In [ ]:
import io
import os
import numpy as np
import pandas as pd
import gzip


def get_vcf_names(vcf_path):
    with gzip.open(vcf_path, "/content/drive/My Drive/002_snps") as ifile:
          for line in ifile:
            if line.startswith("#CHROM"):
                vcf_names = [x for x in line.split('\t')]
                break
    ifile.close()
    return vcf_names


def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

def in_between(position, relevent):
    appears = False
    for i in range(len(relevent)):
        row = relevent.iloc[i]
        if (position >= relevent.iloc[i].start) and (position <= relevent.iloc[i].end):
            appears = True
    return appears

def main():


    genes = pd.read_csv('/content/drive/My Drive/gene_list.csv')
    files = os.listdir('/content/drive/My Drive/002_snps')


    for vcf_file in files:
        file_name = "YOUR_PATH_TO_VCFS" + vcf_file

        output_file = open('log.txt','a')
        output_file.write(file_name)
        output_file.close()
        names = get_vcf_names(file_name)
        vcf = pd.read_csv(file_name, compression='gzip', comment='#', chunksize=10000, delim_whitespace=True, header=None, names=names)
        vcf = pd.concat(vcf, ignore_index=True)

        start = vcf_file.find("ADNI_ID.") + len("ADNI_ID.")
        end = vcf_file.find("output.vcf")
        substring = vcf_file[start:end]
        relevent = genes[genes["chrom"] == substring]
        relevent = relevent.reset_index()

        positions = vcf["POS"]


        indexes = []
        for i in range(len(positions)):

            boo = in_between(positions[i], relevent)
            if i % 500 == 0:
                output_file = open('log.txt','a')
                output_file.write(" " + str(boo) + " ")
                output_file.close()
            if boo:
                indexes.append(i)

        if len(indexes) != 0:
            df = vcf.iloc[indexes]
            df.to_pickle(vcf_file[:-4] + ".pkl")


if __name__ == '__main__':
    main()


Compile all the genetic files together

In [ ]:
import io
import os
import numpy as np
import pandas as pd


def main():



    files = os.listdir("YOUR_PATH_TO_FILTERED_VCFS")
    diag = pd.read_csv("YOUR_PATH_TO_DIAGNOSIS_TABLE")[["index", "Group"]]

    vcfs = []

    for vcf_file in files:
        file_name = "YOUR_PATH_TO_FILTERED_VCFS" + vcf_file

        vcf = pd.read_pickle(file_name)

        vcf = vcf.drop(['#CHROM', 'POS', 'ID','REF','ALT','QUAL','FILTER','INFO', 'FORMAT'], axis=1)
        vcf = vcf.T
        vcf.reset_index(level=0, inplace=True)
        vcf["index"] = vcf["index"].str.replace("s", "S").str.replace("\n", "")
        merged = diag.merge(vcf, on = "index")
        merged = merged.rename(columns={"index": "subject"})
        d = {'0/0': 0, '0/1': 1, '1/0': 1,  '1/1': 2, "./.": 3}
        cols = list(set(merged.columns) - set(["subject", "Group"]))
        for col in cols:
            merged[col] = merged[col].str[:3].replace(d)
            idx = cols.index(col)
            if idx % 500 == 0:
                output_file = open('log_clean.txt','a')
                output_file.write("Percent done: " + str((idx/len(cols))*100) + "\n")
                output_file.close()

        merged.to_pickle(vcf_file + "clean.pkl")

        vcf = vcf.groupby('index', group_keys=False).apply(lambda x: x.loc[x.Group.idxmax()])

        vcfs.append(vcf)

    vcf = pd.concat(vcfs, ignore_index=True)
    vcf = vcf.drop_duplicates()
    vcf.to_pickle("all_vcfs.pkl")




if __name__ == '__main__':
    main()


Further reduce the number of features through with a Random Forest algorithm.

In [ ]:
#reading all the SNP files
vcf = pd.read_pickle("all_vcfs.pkl")

In [ ]:
#reading in the diagnosis data
m = pd.read_csv("diagnosis_full.csv").drop("Unnamed: 0", axis=1).rename(columns = {"Subject":"subject", "GROUP": "label"})

In [ ]:
#making sure all the diagnosis agree
m = m[m["label"] != -1]

In [ ]:
#merging SNPs with diagnosis
vcf = vcf.merge(m[["subject", "label"]], on = "subject")

In [ ]:
vcf = vcf.drop_duplicates()

In [ ]:
#reading in the overlap test set
ts = pd.read_csv("overlap_test_set.csv")

#removing ids from the overlap test set, saving it as a new variable
vcf1 = vcf[~vcf["subject"].isin(list(ts["subject"].values))]

In [ ]:
# Using Random Forest to reduce feature dimensions
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))

In [ ]:
cols = list(set(vcf1.columns) - set(["subject", "Group", "label"]))
X = vcf1[cols].values.astype(int)
y = vcf1["label"].astype(int).values

for i in range(len(y)):
    y[i] = y[i]-1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)


In [ ]:
#fitting random forest only on the training data so that there is not influence on the testing performance
sel.fit(X_train, y_train)

In [ ]:
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

In [ ]:
print(selected_feat)

In [ ]:
l = ["label", "subject", "Group"]
l.extend(selected_feat)

In [ ]:
#saving the features with the old dataframe so that the overlap test set can still be used when combining all data
vcf[l].to_pickle("vcf_select.pkl")


In [ ]:
X_train.to_pickle("X_train_vcf.pkl")
y_train.to_pickle("y_train_vcf.pkl")

X_test.to_pickle("X_test_vcf.pkl")
y_test.to_pickle("y_test_vcf.pkl")

##   Model/Training/Evaluation


##Citation

Golovanevsky, Michal, Carsten Eickhoff, and Ritambhara Singh. "Multimodal attention-based deep learning for Alzheimer’s disease diagnosis." Journal of the American Medical Informatics Association 29, no. 12 (2022): 2014–2022.

**Train and evaluate a uni-modal model baseline for processed clinical data**

In [ ]:
import pandas as pd
import numpy as np
import os
import random
import tensorflow as tf

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential



def reset_random_seeds(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


def main():
    #this is created in the clinical preprocess jupyter notebook
    X_train = pd.read_pickle("X_train_c.pkl")
    y_train = pd.read_pickle("y_train_c.pkl")

    X_test = pd.read_pickle("X_test_c.pkl")
    y_test = pd.read_pickle("y_test_c.pkl")

    acc = []
    f1 = []
    precision = []
    recall = []
    seeds = random.sample(range(1, 200), 5)
    for seed in seeds:
        reset_random_seeds(seed)
        model = Sequential()
        model.add(Dense(128, input_shape = (185,), activation = "relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(64, activation = "relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Dense(50, activation = "relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))

        model.add(Dense(3, activation = "softmax"))

        model.compile(Adam(learning_rate = 0.0001), "sparse_categorical_crossentropy", metrics = ["sparse_categorical_accuracy"])

        model.summary()


        history = model.fit(X_train, y_train,  epochs=100, validation_split=0.1, batch_size=32,verbose=1)

        score = model.evaluate(X_test, y_test, verbose=0)
        print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
        acc.append(score[1])

        test_predictions = model.predict(X_test)
        test_label = to_categorical(y_test,3)

        true_label= np.argmax(test_label, axis =1)

        predicted_label= np.argmax(test_predictions, axis =1)

        cr = classification_report(true_label, predicted_label, output_dict=True)
        precision.append(cr["macro avg"]["precision"])
        recall.append(cr["macro avg"]["recall"])
        f1.append(cr["macro avg"]["f1-score"])

    print("Avg accuracy: " + str(np.array(acc).mean()))
    print("Avg precision: " + str(np.array(precision).mean()))
    print("Avg recall: " + str(np.array(recall).mean()))
    print("Avg f1: " + str(np.array(f1).mean()))
    print("Std accuracy: " + str(np.array(acc).std()))
    print("Std precision: " + str(np.array(precision).std()))
    print("Std recall: " + str(np.array(recall).std()))
    print("Std f1: " + str(np.array(f1).std()))
    print(acc)
    print(precision)
    print(recall)
    print(f1)



    plt.plot(history.history['sparse_categorical_accuracy'])
    plt.plot(history.history['val_sparse_categorical_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    plt.clf()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    #plt.savefig('snp_loss.png')
    plt.show()



if __name__ == '__main__':
    main()

**Train and evaluate a uni-modal model baseline for processed imaging data**

In [ ]:
import os
import random
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import pickle5 as pickle
import matplotlib.pyplot as plt
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from keras.layers import Dense,Dropout,MaxPooling2D, Flatten, Conv2D


def reset_random_seeds(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


def main():

    with open("img_train.pkl", "rb") as fh:
        data = pickle.load(fh)
    X_train_ = pd.DataFrame(data)["img_array"]

    with open("img_test.pkl", "rb") as fh:
        data = pickle.load(fh)
    X_test_ = pd.DataFrame(data)["img_array"]

    with open("img_y_train.pkl", "rb") as fh:
        data = pickle.load(fh)
    y_train = np.array(pd.DataFrame(data)["label"].values.astype(np.float32)).flatten()

    with open("img_y_test.pkl", "rb") as fh:
        data = pickle.load(fh)
    y_test = np.array(pd.DataFrame(data)["label"].values.astype(np.float32)).flatten()


    y_test[y_test == 2] = -1
    y_test[y_test == 1] = 2
    y_test[y_test == -1] = 1

    y_train[y_train == 2] = -1
    y_train[y_train == 1] = 2
    y_train[y_train == -1] = 1


    X_train = []
    X_test = []

    for i in range(len(X_train_)):
        X_train.append(X_train_.values[i])

    for i in range(len(X_test_)):
        X_test.append(X_test_.values[i])


    X_train = np.array(X_train)
    X_test = np.array(X_test)


    acc = []
    f1 = []
    precision = []
    recall = []
    seeds = random.sample(range(1, 200), 5)
    for seed in seeds:
        reset_random_seeds(seed)
        model = Sequential()
        model.add(Conv2D(100, (3, 3),  activation='relu', input_shape=(72, 72, 3)))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.5))
        model.add(Conv2D(50, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.3))
        model.add(Flatten())
        model.add(Dense(3, activation = "softmax"))


        model.compile(Adam(learning_rate = 0.001), "sparse_categorical_crossentropy", metrics = ["sparse_categorical_accuracy"])

        model.summary()


        history = model.fit(X_train, y_train, epochs=50, batch_size=32,validation_split=0.1, verbose=1)

        score = model.evaluate(X_test, y_test, verbose=0)
        print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
        acc.append(score[1])

        test_predictions = model.predict(X_test)
        test_label = to_categorical(y_test,3)

        true_label= np.argmax(test_label, axis =1)

        predicted_label= np.argmax(test_predictions, axis =1)

        cr = classification_report(true_label, predicted_label, output_dict=True)
        precision.append(cr["macro avg"]["precision"])
        recall.append(cr["macro avg"]["recall"])
        f1.append(cr["macro avg"]["f1-score"])

    print("Avg accuracy: " + str(np.array(acc).mean()))
    print("Avg precision: " + str(np.array(precision).mean()))
    print("Avg recall: " + str(np.array(recall).mean()))
    print("Avg f1: " + str(np.array(f1).mean()))
    print("Std accuracy: " + str(np.array(acc).std()))
    print("Std precision: " + str(np.array(precision).std()))
    print("Std recall: " + str(np.array(recall).std()))
    print("Std f1: " + str(np.array(f1).std()))
    print(acc)
    print(precision)
    print(recall)
    print(f1)



if __name__ == '__main__':
    main()


**Train and evaluate a uni-modal model baseline for processed genetic data**

In [ ]:

import numpy as np
import pandas as pd
import os
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout, BatchNormalization
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential

def reset_random_seeds(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


def main():

        #this is created in the genetic preprocess jupyter notebook
        X_train = pd.read_pickle("X_train_vcf.pkl")
        y_train = pd.read_pickle("y_train_vcf.pkl")

        X_test = pd.read_pickle("X_test_vcf.pkl")
        y_test = pd.read_pickle("y_test_vcf.pkl")


        acc = []
        f1 = []
        precision = []
        recall = []
        seeds = random.sample(range(1, 200), 5)

        for seed in seeds:
            reset_random_seeds(seed)
            model = Sequential()
            model.add(Dense(128, input_shape = (15965,), activation = "relu"))
            model.add(Dropout(0.5))
            model.add(Dense(64, activation = "relu"))
            model.add(Dropout(0.5))

            model.add(Dense(32, activation = "relu"))
            model.add(Dropout(0.3))

            model.add(Dense(32, activation = "relu"))
            model.add(Dropout(0.3))


            model.add(Dense(3, activation = "softmax"))

            model.compile(Adam(learning_rate = 0.001), "sparse_categorical_crossentropy", metrics = ["sparse_categorical_accuracy"])


            history = model.fit(X_train, y_train,epochs=50,batch_size=32,validation_split = 0.1, verbose=1)

            score = model.evaluate(X_test, y_test, verbose=0)
            print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
            acc.append(score[1])

            test_predictions = model.predict(X_test)
            test_label = to_categorical(y_test,3)

            true_label= np.argmax(test_label, axis =1)

            predicted_label= np.argmax(test_predictions, axis =1)

            cr = classification_report(true_label, predicted_label, output_dict=True)
            precision.append(cr["macro avg"]["precision"])
            recall.append(cr["macro avg"]["recall"])
            f1.append(cr["macro avg"]["f1-score"])

        print("Avg accuracy: " + str(np.array(acc).mean()))
        print("Avg precision: " + str(np.array(precision).mean()))
        print("Avg recall: " + str(np.array(recall).mean()))
        print("Avg f1: " + str(np.array(f1).mean()))
        print("Std accuracy: " + str(np.array(acc).std()))
        print("Std precision: " + str(np.array(precision).std()))
        print("Std recall: " + str(np.array(recall).std()))
        print("Std f1: " + str(np.array(f1).std()))
        print(acc)
        print(precision)
        print(recall)
        print(f1)



if __name__ == '__main__':
    main()


**Train and evaluate the multimodal architecture**

In [ ]:
import os
import random
import gc, numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import compute_class_weight
import tensorflow as tf
from keras.models import Model
from keras import backend as K
from keras.layers import Input, Dense, Dropout,Flatten, BatchNormalization, Conv2D, MultiHeadAttention, concatenate
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import seaborn as sns
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve


config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

def make_img(t_img):
    img = pd.read_pickle(t_img)
    img_l = []
    for i in range(len(img)):
        img_l.append(img.values[i][0])

    return np.array(img_l)


def reset_random_seeds(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


def create_model_snp():

    model = Sequential()
    model.add(Dense(200,  activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(100, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(50, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    return model

def create_model_clinical():

    model = Sequential()
    model.add(Dense(200,  activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(100, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(50, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    return model

def create_model_img():



    model = Sequential()
    model.add(Conv2D(72, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    return model


def plot_classification_report(y_tru, y_prd, mode, learning_rate, batch_size,epochs, figsize=(7, 7), ax=None):

    plt.figure(figsize=figsize)

    xticks = ['precision', 'recall', 'f1-score', 'support']
    yticks = ["Control", "Moderate", "Alzheimer's" ]
    yticks += ['avg']

    rep = np.array(precision_recall_fscore_support(y_tru, y_prd)).T
    avg = np.mean(rep, axis=0)
    avg[-1] = np.sum(rep[:, -1])
    rep = np.insert(rep, rep.shape[0], avg, axis=0)

    sns.heatmap(rep,
                annot=True,
                cbar=False,
                xticklabels=xticks,
                yticklabels=yticks,
                ax=ax, cmap = "Blues")

    plt.savefig('report_' + str(mode) + '_' + str(learning_rate) +'_' + str(batch_size)+'_' + str(epochs)+'.png')



def calc_confusion_matrix(result, test_label,mode, learning_rate, batch_size, epochs):
    test_label = to_categorical(test_label,3)

    true_label= np.argmax(test_label, axis =1)

    predicted_label= np.argmax(result, axis =1)

    n_classes = 3
    precision = dict()
    recall = dict()
    thres = dict()
    for i in range(n_classes):
        precision[i], recall[i], thres[i] = precision_recall_curve(test_label[:, i],
                                                            result[:, i])


    print ("Classification Report :")
    print (classification_report(true_label, predicted_label))
    cr = classification_report(true_label, predicted_label, output_dict=True)
    return cr, precision, recall, thres



def cross_modal_attention(x, y):
    x = tf.expand_dims(x, axis=1)
    y = tf.expand_dims(y, axis=1)
    a1 = MultiHeadAttention(num_heads = 4,key_dim=50)(x, y)
    a2 = MultiHeadAttention(num_heads = 4,key_dim=50)(y, x)
    a1 = a1[:,0,:]
    a2 = a2[:,0,:]
    return concatenate([a1, a2])


def self_attention(x):
    x = tf.expand_dims(x, axis=1)
    attention = MultiHeadAttention(num_heads = 4, key_dim=50)(x, x)
    attention = attention[:,0,:]
    return attention


def multi_modal_model(mode, train_clinical, train_snp, train_img):

    in_clinical = Input(shape=(train_clinical.shape[1]))

    in_snp = Input(shape=(train_snp.shape[1]))

    in_img = Input(shape=(train_img.shape[1], train_img.shape[2], train_img.shape[3]))

    dense_clinical = create_model_clinical()(in_clinical)
    dense_snp = create_model_snp()(in_snp)
    dense_img = create_model_img()(in_img)



    ########### Attention Layer ############

    ## Cross Modal Bi-directional Attention ##

    if mode == 'MM_BA':

        vt_att = cross_modal_attention(dense_img, dense_clinical)
        av_att = cross_modal_attention(dense_snp, dense_img)
        ta_att = cross_modal_attention(dense_clinical, dense_snp)

        merged = concatenate([vt_att, av_att, ta_att, dense_img, dense_snp, dense_clinical])




    ## Self Attention ##
    elif mode == 'MM_SA':

        vv_att = self_attention(dense_img)
        tt_att = self_attention(dense_clinical)
        aa_att = self_attention(dense_snp)

        merged = concatenate([aa_att, vv_att, tt_att, dense_img, dense_snp, dense_clinical])

    ## Self Attention and Cross Modal Bi-directional Attention##
    elif mode == 'MM_SA_BA':

        vv_att = self_attention(dense_img)
        tt_att = self_attention(dense_clinical)
        aa_att = self_attention(dense_snp)

        vt_att = cross_modal_attention(vv_att, tt_att)
        av_att = cross_modal_attention(aa_att, vv_att)
        ta_att = cross_modal_attention(tt_att, aa_att)

        merged = concatenate([vt_att, av_att, ta_att, dense_img, dense_snp, dense_clinical])


    ## No Attention ##
    elif mode == 'None':

        merged = concatenate([dense_img, dense_snp, dense_clinical])

    else:
        print ("Mode must be one of 'MM_SA', 'MM_BA', 'MU_SA_BA' or 'None'.")
        return


    ########### Output Layer ############

    output = Dense(3, activation='softmax')(merged)
    model = Model([in_clinical, in_snp, in_img], output)

    return model



def train(mode, batch_size, epochs, learning_rate, seed):


    train_clinical = pd.read_csv("X_train_clinical.csv").drop("Unnamed: 0", axis=1).values
    test_clinical= pd.read_csv("X_test_clinical.csv").drop("Unnamed: 0", axis=1).values


    train_snp = pd.read_csv("X_train_snp.csv").drop("Unnamed: 0", axis=1).values
    test_snp = pd.read_csv("X_test_snp.csv").drop("Unnamed: 0", axis=1).values


    train_img= make_img("X_train_img.pkl")
    test_img= make_img("X_test_img.pkl")


    train_label= pd.read_csv("y_train.csv").drop("Unnamed: 0", axis=1).values.astype("int").flatten()
    test_label= pd.read_csv("y_test.csv").drop("Unnamed: 0", axis=1).values.astype("int").flatten()

    reset_random_seeds(seed)
    class_weights = compute_class_weight(class_weight = 'balanced',classes = np.unique(train_label),y = train_label)
    d_class_weights = dict(enumerate(class_weights))

    # compile model #
    model = multi_modal_model(mode, train_clinical, train_snp, train_img)
    model.compile(optimizer=Adam(learning_rate = learning_rate), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])


    # summarize results
    history = model.fit([train_clinical,
                         train_snp,
                         train_img],
                        train_label,
                        epochs=epochs,
                        batch_size=batch_size,
                        class_weight=d_class_weights,
                        validation_split=0.1,
                        verbose=1)



    score = model.evaluate([test_clinical, test_snp, test_img], test_label)

    acc = score[1]
    test_predictions = model.predict([test_clinical, test_snp, test_img])
    cr, precision_d, recall_d, thres = calc_confusion_matrix(test_predictions, test_label, mode, learning_rate, batch_size, epochs)



    plt.clf()
    plt.plot(history.history['sparse_categorical_accuracy'])
    plt.plot(history.history['val_sparse_categorical_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    plt.savefig('accuracy_' + str(mode) + '_' + str(learning_rate) +'_' + str(batch_size)+'.png')
    plt.clf()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    plt.savefig('loss_' + str(mode) + '_' + str(learning_rate) +'_' + str(batch_size)+'.png')
    plt.clf()




    # release gpu memory #
    K.clear_session()
    del model, history
    gc.collect()


    print ('Mode: ', mode)
    print ('Batch size:  ', batch_size)
    print ('Learning rate: ', learning_rate)
    print ('Epochs:  ', epochs)
    print ('Test Accuracy:', '{0:.4f}'.format(acc))
    print ('-'*55)

    return acc, batch_size, learning_rate, epochs, seed


if __name__=="__main__":

    m_a = {}
    seeds = random.sample(range(1, 200), 5)
    for s in seeds:
        acc, bs_, lr_, e_ , seed= train('MM_SA_BA', 32, 50, 0.001, s)
        m_a[acc] = ('MM_SA_BA', acc, bs_, lr_, e_, seed)
    print(m_a)
    print ('-'*55)
    max_acc = max(m_a, key=float)
    print("Highest accuracy of: " + str(max_acc) + " with parameters: " + str(m_a[max_acc]))


# Results
The models and evaluation metrics were all carefully implemented as explained in the original paper. However, the exact datasets that needed to be downloaded to properly execute the study were nearly impossible to figure out. There were over 1000 files to manually screen through (all with naming convention different from the original study). Furthermore, they were not from the time range that the study was conducted (i.e. I only had access to 2024 data). As such, the hypotheses I layed out where difficult to verify.

# Discussion

The paper is reproducible, though acquiring the data is a struggle unless you continously follow up with the DADNI.

What Is Easy: implementing the model.

What Is Difficult: Acquiring the data.Ensuring accurate pre-processing of three types of data (clinical, imaging, genetics)

Suggestion: Please provide pre-processed data in the materials section of the paper.

Next phase: I would refine the multimodal modal and optimize hyperparameters so its more accurate in detecting Alzherimers.


# References

1.   Golovanevsky, Michal, Carsten Eickhoff, and Ritambhara Singh. "Multimodal attention-based deep learning for Alzheimer’s disease diagnosis." Journal of the American Medical Informatics Association 29, no. 12 (2022): 2014–2022.

